In [1]:
import numpy as np
import pandas as pd
import warnings
import pickle
import json
import requests

warnings.filterwarnings('ignore')

In [2]:
with open('drug-event-0001-of-0035.json') as json_file:
    data = json.load(json_file)

In [3]:
data['results'][0]['patient']['drug'][0]['openfda']
#len(obs["patient"]["drug"])

{'manufacturer_name': ['Novo Nordisk'],
 'unii': ['839I73S42A'],
 'product_type': ['HUMAN PRESCRIPTION DRUG'],
 'rxcui': ['1598268', '897122'],
 'spl_set_id': ['3946d389-0926-4f77-a708-0acb8153b143'],
 'route': ['SUBCUTANEOUS'],
 'generic_name': ['LIRAGLUTIDE'],
 'brand_name': ['SAXENDA'],
 'product_ndc': ['0169-2800'],
 'substance_name': ['LIRAGLUTIDE'],
 'spl_id': ['88ec60c6-7a45-433e-b653-429a8d6179a0'],
 'application_number': ['NDA206321'],
 'package_ndc': ['0169-2800-90', '0169-2800-15', '0169-2800-97']}

In [5]:
count_of_received = pickle.load(open('count_of_received.p', 'rb'))

In [6]:
def define_time_intervals(json_results):
    count = 0
    date_cutoffs = []
    for i in json_results:
        if count == 0:
            first_time = i['time']
        if (count + i['count']) < 100:
            count += i['count']
        else:
            date_cutoffs.append((first_time, i['time']))
            count = 0
    return date_cutoffs

In [7]:
time_intervals = define_time_intervals(count_of_received)

In [8]:
def get_company_data(data):
    company_list = []
    inter = data['results']
    for i in range(len(data['results'])):
        obs = inter[i]
        patient_drug_nums = len(obs['patient']['drug'])
        if patient_drug_nums == 1:
            try:
                manufact = obs['patient']['drug'][0]['openfda']['manufacturer_name']
                if len(manufact) == 1:
                    company_list.append(manufact)
                else:
                    company_list.append(np.nan)
            except:
                    company_list.append(np.nan)
        else:
            company_list.append(np.nan)
    
    return pd.Series(company_list)

In [9]:
def get_json_data_by_company(time_intervals):
    all_data = []
    for time in time_intervals:
        url = f'https://api.fda.gov/drug/event.json?search=receivedate:[{time[0]}+TO+{time[1]}]&limit=99'
        data = requests.get(url).json()
        all_data.append(get_company_data(data))
    series = pd.concat(all_data)
    return series

In [65]:
#companies = get_json_data_by_company(time_intervals)

In [69]:
#pickle.dump(companies, open('pickles/company_info.p', 'wb'))

In [70]:
final_dataframe = pickle.load(open('final_dataframe.p', 'rb'))

In [78]:
final_dataframe['company'] = companies.values

In [80]:
#pickle.dump(final_dataframe, open('pickles/final_dataframe.p', 'wb'))

In [11]:
len(final_dataframe[final_dataframe['company'].isna()])

NameError: name 'final_dataframe' is not defined